# Comment fonctionne basiquement un apprentissage (Machine learning)

Gestion des données et représentation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Optimisation via scipy

In [ ]:
from scipy.optimize import minimize

In [ ]:
# Gestion de la taille des figures
plt.rcParams["figure.figsize"] = (14, 10)

## Génération de l'échantillon

In [ ]:
a, b = -1, 1

In [ ]:
def cible(x):
    return np.exp(-20 * x ** 2)

In [ ]:
xs = np.linspace(a, b, 200) # 200 points entre - 1 et 1
ys = cible(xs) # Output
print(xs)

In [ ]:
plt.plot(xs, ys)

In [ ]:
nb_points = 20
points = np.random.uniform(a, b, size=(nb_points))

In [ ]:
print(points)

In [ ]:
valeurs = cible(points) # Output de ces 20 points

In [ ]:
plt.plot(xs, ys, color="blue", label="cible")
plt.scatter(points, valeurs, color="red", label="echantillon")
plt.legend()

On a notre fonction cible en bleu (l'objectif est d'apprendre cette fonction), et pour cela on a nos données d'entraînement en rouge.

## Apprentissage

On peut apprendre la fonction cible en faisant une interpolation polynomiale de Lagrange.

Basiquement, on note l'interpolation polynomiale de Lagrange pour un point $i$ de la façon suivante : 


$$ P(x_i) = a_nx^n + a_{n-1}x^{n-1} + ... + a_2x^2 + a_1x^1 + a_0 $$

Soit $y_i$ la valeur de l'output, il faut donc que $P(x_i) = y_i$.

Il faut donc déterminer les coefficients $a_n, ... , a_0$ de telle façon à minimiser l'erreur de prédiction. On cherche alors : 

$$ Min(\sum^{n}_{i = 0} (P(x_i) - y_i)^2)$$ 

In [ ]:
def erreur(coefficients):
    return sum(
        (
            sum( 
                coefficient * point ** k 
                for k, coefficient in enumerate(coefficients)
            ) 
            - valeur
        ) ** 2 
        for point, valeur in zip(points, valeurs)
    )

In [ ]:
degres = 5
resultat = minimize(erreur, np.zeros(shape=(degres + 1)))

In [ ]:
print(resultat)

In [ ]:
coefficients_finaux = resultat.x
print(coefficients_finaux)

In [ ]:
def evaluation_polynomial(x, coefficients):
    return sum(
        coefficient * x ** k 
        for k, coefficient in enumerate(coefficients)
    )

In [ ]:
plt.plot(xs, ys, color="blue", label="cible")
plt.scatter(points, valeurs, color="red", label="echantillon")
plt.plot(
    xs, 
    evaluation_polynomial(xs, coefficients_finaux), 
    color="green", 
    label=f"apprise avec d={degres}"
)
plt.legend()

In [ ]:
erreur(coefficients_finaux)

## Reformatage du code

In [ ]:
def apprentissage(degres):
    resultat = minimize(erreur, np.zeros(shape=(degres + 1)))
    def evaluation(x):
        return sum(
            coefficient * x ** k 
        for k, coefficient in enumerate(resultat.x)
    )
    return evaluation

In [ ]:
plt.plot(xs, ys, color="blue", label="cible")
plt.scatter(points, valeurs, color="red", label="echantillon")
for degres in (5, 8, 11):
    evaluation = apprentissage(degres)
    plt.plot(
        xs, 
        evaluation(xs), 
        label=f"apprise avec d={degres}"
)
plt.legend()
ax = plt.gca()
ax.set_ylim([-0.2, 1.2])

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=3)
xs = np.linspace(a, b, 200)
ys = cible(xs)
for degres, ax in zip((5, 7, 9, 11, 20, 30), axs.flatten()):
    ax.plot(xs, ys, color="blue", label="cible")
    ax.scatter(points, valeurs, color="red", label="echantillon")  
    evaluation = apprentissage(degres)
    ax.plot(
        xs, 
        evaluation(xs), 
        label=f"apprise",
        color="green",
)
    ax.set_ylim(-0.1, 1.1)
    ax.set_title(f"degres={degres}")
    ax.legend()

### On peut refaire cet apprentissage en séparant l'échantillon en deux parties (train et test)

In [ ]:
a, b = -1, 1
def cible(x):
    return np.exp(-20 * x ** 2)

In [ ]:
points_apprentissage = np.random.uniform(a, b, size=(20))
points_test = np.random.uniform(a, b, size=(10))

In [ ]:
valeurs_apprentissage = cible(points_apprentissage)
valeurs_test = cible(points_test)

In [ ]:
def erreur_apprentissage(coefficients):
    return sum(
        (
            sum( 
                coefficient * point ** k
                for k, coefficient in enumerate(coefficients)
            )
            - valeur
        ) ** 2 
        for point, valeur in zip(points_apprentissage, valeurs_apprentissage)
    )

In [ ]:
def erreur_test(coefficients):
    return sum(
        (
            sum( 
                coefficient * point ** k
                for k, coefficient in enumerate(coefficients)
            )
            - valeur
        ) ** 2 
        for point, valeur in zip(points_test, valeurs_test)
    )

In [ ]:
def apprentissage(degres):
    resultat = minimize(fun=erreur_apprentissage, x0=np.zeros(shape=(degres + 1)))
    return resultat.x

In [ ]:
erreurs_apprentissage = list()
erreurs_test = list()
for degres in range(1, 21):
    coefficients_optimaux = apprentissage(degres)
    erreurs_apprentissage.append(erreur_apprentissage(coefficients_optimaux))
    erreurs_test.append(erreur_test(coefficients_optimaux))

In [ ]:
plt.semilogy(erreurs_apprentissage, label="erreurs sur la partie apprise")
plt.semilogy(erreurs_test, label="erreurs sur la partie test")
plt.legend()

In [ ]:
class Echantillon:
    def __init__(
        self, 
        cible, 
        nb_apprentissage, 
        nb_validation,
        a,
        b,
    ):
        self.nb_validation = nb_validation
        self.nb_apprentissage = nb_apprentissage
        self.xs = np.linspace(a, b, 200)
        self.cible = cible
        self.points_app = np.random.uniform(a, b, size=(self.nb_apprentissage))
        self.points_val = np.random.uniform(a, b, size=(self.nb_validation))
        self.valeurs_app = self.cible(self.points_app)
        self.valeurs_val = self.cible(self.points_val)
        
    def erreur_app(self, polynome):
        return sum(
            (polynome(point) - valeur) ** 2
            for point, valeur in zip(self.points_app, self.valeurs_app)
        )
            
    def erreur_val(self, polynome):
        return sum(
            (polynome(point) - valeur) ** 2 
            for point, valeur in zip(self.points_val, self.valeurs_val)
        ) / self.nb_validation
    
    def affichage(self, ax):
        ax.plot(xs, self.cible(xs), label="cible")
        ax.scatter(self.points_app, self.valeurs_app, label="echantillon_apprentissage")
        ax.scatter(self.points_val, self.valeurs_val, label="echantillon_validation") 

In [ ]:
class Modele:
    def __init__(self, degres, echantillon):
        self.degres = degres
        self.echantillon = echantillon
        
    def apprentissage(self):
        def evaluation(coefficients):
            return self.echantillon.erreur_app(
                lambda x: sum(coefficient * x ** k for k, coefficient in enumerate(coefficients))
            )
        resultat = minimize(
            fun=evaluation,
            x0=np.zeros(shape=(self.degres))
        )
        self.coefficients_optimaux = resultat.x
        
    def affichage(self, ax):
        self.valeurs = sum(
            coefficient_optimal * self.echantillon.xs ** k
            for k, coefficient_optimal in enumerate(self.coefficients_optimaux)
        )
        ax.plot(
            self.echantillon.xs, 
            self.valeurs, 
            label=f"appris d={self.degres}"
        )
        
    def affichage_erreur_val(self):
        erreur = self.echantillon.erreur_val(
            lambda x: sum(coefficient * x ** k for k, coefficient in enumerate(self.coefficients_optimaux))
        ) 
        return print(f"Erreur en test avec d = {self.degres} : {erreur}")
        

In [ ]:
ech = Echantillon(
    cible=lambda x: np.exp(- 20 * x ** 2),
    a=-1,
    b=1,
    nb_apprentissage=50,
    nb_validation=30,
)
fig, ax = plt.subplots()
ax.set_ylim(-0.1, 1.1)
ech.affichage(ax)
m5 = Modele(degres=5, echantillon=ech)
m5.apprentissage()
m5.affichage(ax)
m5.affichage_erreur_val()
m9 = Modele(degres=9, echantillon=ech)
m9.apprentissage()
m9.affichage(ax)
m9.affichage_erreur_val()
m11 = Modele(degres=11, echantillon=ech)
m11.apprentissage()
m11.affichage(ax)
m11.affichage_erreur_val()
m50 = Modele(degres=50, echantillon=ech)
m50.apprentissage()
m50.affichage(ax)
m50.affichage_erreur_val()
ax.legend()